# Final Project Phase 2 Summary
This Jupyter Notebook (.ipynb) will serve as the skeleton file for your submission for Phase 2 of the Final Project. Complete all sections below as specified in the instructions for the project, covering all necessary details. We will use this to grade your individual code (Do this whether you are in a group or not). Good luck! <br><br>

Note: To edit a Markdown cell, double-click on its text.

## Jupyter Notebook Quick Tips
Here are some quick formatting tips to get you started with Jupyter Notebooks. This is by no means exhaustive, and there are plenty of articles to highlight other things that can be done. We recommend using HTML syntax for Markdown but there is also Markdown syntax that is more streamlined and might be preferable. 
<a href = "https://towardsdatascience.com/markdown-cells-jupyter-notebook-d3bea8416671">Here's an article</a> that goes into more detail. (Double-click on cell to see syntax)

# Heading 1
## Heading 2
### Heading 3
#### Heading 4
<br>
<b>BoldText</b> or <i>ItalicText</i>
<br> <br>
Math Formulas: $x^2 + y^2 = 1$
<br> <br>
Line Breaks are done using br enclosed in < >.
<br><br>
Hyperlinks are done with: <a> https://www.google.com </a> or 
<a href="http://www.google.com">Google</a><br>

# Data Collection and Cleaning


Transfer/update the data collection and cleaning you created for Phase I below. You may include additional cleaning functions if you have extra datasets. 


## Downloaded Dataset Requirement



In [ ]:
import json
import csv
from pprint import pprint

def data_parser(filename):
    python_dict=json.load(open(filename))
    d_data=python_dict["data"]

    county_list=[]

    for county in d_data:
        if county not in county_list:
            county_list.append(county[9])

    county_list_final=sorted(list(set(county_list)),key=lambda x:x[0])


    first_case_index=[]
    date_of_fcase=[]

    for county in range(len(d_data)):
        if int(d_data[county][11]) != 0:
            if int(d_data[county-1][11])==0:
                first_case_index.append(county+2)
                date_of_fcase.append(d_data[county][8][0:10])


    last_case_index=[]
    date_of_lcase=[]
    for county in range(len(d_data)):
        if (d_data[county][8]) == "2020-10-25T00:00:00":
            last_case_index.append(county+2)
            date_of_lcase.append(d_data[county][8][0:10])



    cummulative_cases_per_county=[]
    for county in range(len(d_data)):
        if d_data[county][8]=="2020-10-25T00:00:00":
            cummulative_cases_per_county.append(int(d_data[county][11]))


    avg_rate_c=[]

    total_tests_performed=[]
    for county in range(len(d_data)):
        if d_data[county][8]=="2020-10-25T00:00:00":
            total_tests_performed.append(int(d_data[county][13]))


    for num in range(len(cummulative_cases_per_county)):
        avg_rate_c.append(round(((cummulative_cases_per_county[num])/(last_case_index[num]-first_case_index[num]+1)),2))



    final_dict={}
    counter=-1
    for county in county_list_final:
        counter+=1
        if county not in final_dict.keys():
            final_dict[county]={"Date of First Case":date_of_fcase[counter],"Date of Last Case":date_of_lcase[counter],
            "Total Cases":cummulative_cases_per_county[counter],"Average Number Cases per Day":avg_rate_c[counter],
            "Total Tests Performed":total_tests_performed[counter]}


    a_list = []
    a_list.append(["County", "Date of First Case", "Date of Last Case", "Total Cases", "Average Number of Cases per Day", "Total Tests Performed"])

    for key, value in final_dict.items():
        counties = [key]
        for k, v in value.items():
            counties.append(v)
        a_list.append(counties)

    with open("NY_Covid19_Data.csv", "w") as fout:
        writer = csv.writer(fout, lineterminator = "\n")
        writer.writerows(a_list)

    pprint(a_list)
    print()
    
    return 'Data has been exported to "NY_Covid19_Data.csv"'

############ Function Call ############
pprint(data_parser("Ny_Statewide_Covid19.json"))

## Downloaded Dataset Requirement #2



In [ ]:
import csv
from pprint import pprint
import pandas as pd
import numpy as np

def data_parser2(filename):
    with open(filename) as f:
        reader=csv.reader(f,delimiter=",",quotechar='"')
        reader_list=list(reader)

        nyc_list=[]
        for city in reader_list:
            if city[7]=="New York":
                nyc_list.append([city[1],city[8],city[9],city[10]])

        return_list = [["All Ages", "0-17 years", "18-64 years", "65+ years"]]

        date_list = []
        for sublist in nyc_list:
            if sublist[0] not in date_list:
                date_list.append(sublist[0])


        for date in date_list:
            deaths = []
            for sublist in nyc_list:
                if date == sublist[0] and sublist[1] == "All Ages":
                    deaths.append(sublist[2])
                elif date == sublist[0] and sublist[1] == "0-17 years":
                    deaths.append(sublist[2])
                elif date == sublist[0] and sublist[1] == "18-64 years":
                    deaths.append(sublist[2])
                elif date == sublist[0] and sublist[1] == "65+ years":
                    deaths.append(sublist[2])
            return_list.append(deaths)

        for sublist in return_list:
            for value in sublist:
                if value == '':
                    sublist[sublist.index(value)] = np.nan


        deaths_arr = np.array(return_list)

        df = pd.DataFrame(data = deaths_arr[1:], columns = deaths_arr[0], index = date_list)

        df.to_csv("NY_Covid_Deaths.csv", index = True)

        pprint(df)
        print()

        return 'Data has been exported to "NY_Covid_Deaths.csv"'
        
  ############ Function Call ############
pprint(data_parser2("COVID_Death.csv"))

## Web Collection Requirement \#1


In [ ]:
import requests; from bs4 import BeautifulSoup; from pprint import pprint; import csv

def web_parser1(filename):

    resp = requests.get(filename)
    soup = BeautifulSoup(resp.text, "html.parser")
    name = soup.find_all("td")
    county_list = []
    income_list = []
    for item in name:
        if name.index(item) % 2 == 1:
            income_list.append(item)
        else:
            county_list.append(item)

    counties = []
    incomes = []


    for county in county_list:
        counties.append(str(county)[4:-5])

    for income in income_list:
        incomes.append(str(income)[18:-5])

    median_income = []

    for income in incomes:
        if "," in income:
            income = income.replace(",", "")
            median_income.append(int(income))

    income_dict = {}

    for i in range(len(counties)):
        income_dict[counties[i]] = median_income[i]

    sorted_incomes = sorted(median_income, reverse = True)

    return_list = []

    for income in sorted_incomes:
        for key, value in income_dict.items():
            if value == income:
                return_list.append([key, value])

    return_list.insert(0, ["County", "Median Income", "Percent of Population with High School Diploma", "Percent of Population with Bachelor's Degree"])

    with open("income_education.csv", "w") as fout:
        writer = csv.writer(fout, lineterminator = "\n")
        writer.writerows(return_list)

    pprint(return_list)
    print()

    return 'Data has been exported to "income_education.csv"'

############ Function Call ############
pprint(web_parser1("https://www.indexmundi.com/facts/united-states/quick-facts/new-york/median-household-income#table"))

## Web Collection Requirement #2

In [ ]:
import requests; from bs4 import BeautifulSoup; from pprint import pprint; import csv

def web_parser2(filename):

    resp = requests.get(filename)
    soup = BeautifulSoup(resp.text, "html.parser")
    name = soup.find_all("td")

    county_list = []
    percent_list = []

    for item in name:
        if name.index(item) % 2 == 1:
            percent_list.append(item)
        else:
            county_list.append(item)

    for i in range(len(county_list)):
        county_list[i] = str(county_list[i])[4:-5]

    for i in range(len(percent_list)):
        percent_list[i] = float(str(percent_list[i])[18:-5])

    high_school_grad_dict = {}

    for i in range(len(county_list)):
        high_school_grad_dict[county_list[i]] = percent_list[i]

    with open("income_education.csv") as f:
        reader = csv.reader(f, delimiter = ",", quotechar = '"')
        readerlist = list(reader)

    for county in readerlist[1:]:
        county.append(high_school_grad_dict[county[0]])

    with open("income_education.csv", "w") as fout:
        writer = csv.writer(fout, lineterminator = "\n")
        writer.writerows(readerlist)

    pprint(readerlist)
    print()

    return 'Data has been exported to "income_education.csv"'

############ Function Call ############
pprint(web_parser2("https://www.indexmundi.com/facts/united-states/quick-facts/new-york/percent-of-people-25-years-and-over-with-high-school-degree-or-higher#table"))

## Web Collection Requirement #3

In [ ]:
import requests; from bs4 import BeautifulSoup; from pprint import pprint; import csv

def web_parser3(filename):

    resp = requests.get(filename)
    soup = BeautifulSoup(resp.text, "html.parser")
    name = soup.find_all("td")

    county_list = []
    percent_list = []

    for item in name:
        if name.index(item) % 2 == 1:
            percent_list.append(item)
        else:
            county_list.append(item)

    for i in range(len(county_list)):
        county_list[i] = str(county_list[i])[4:-5]

    for i in range(len(percent_list)):
        percent_list[i] = float(str(percent_list[i])[18:-5])

    college_grad_dict = {}

    for i in range(len(county_list)):
        college_grad_dict[county_list[i]] = percent_list[i]

    with open("income_education.csv") as f:
        reader = csv.reader(f, delimiter = ",", quotechar = '"')
        readerlist = list(reader)

    for county in readerlist[1:]:
        county.append(college_grad_dict[county[0]])

    with open("income_education.csv", "w") as fout:
        writer = csv.writer(fout, lineterminator = "\n")
        writer.writerows(readerlist)

    pprint(readerlist)
    print()

    return 'Data has been exported to "income_education.csv"'


############ Function Call ############
pprint(web_parser3("https://www.indexmundi.com/facts/united-states/quick-facts/new-york/percent-of-people-25-years-and-over-with-bachelors-degree-or-higher#table"))

## Web Collection Requirement #4

In [ ]:
import requests; from bs4 import BeautifulSoup; from pprint import pprint; import csv; import re; import numpy as np

def web_parser4(filename):

    resp = requests.get(filename)
    soup = BeautifulSoup(resp.text, "html.parser")
    name = soup.find_all("td")

    county_list = []
    population_list = []

    for item in name:
        if name.index(item) % 2 == 1:
            population_list.append(item)
        else:
            county_list.append(item)

    for i in range(len(county_list)):
        county_list[i] = str(county_list[i])[:-10]

    indexes = []
    for county in county_list:
        indexes.append([i.start() for i in re.finditer(">", county)])

    for i in range(len(county_list)):
        county_list[i] = county_list[i][(indexes[i][1]) + 1:]

    for i in range(len(population_list)):
        population_list[i] = int(str(population_list[i])[18:-5].replace(",", ""))

    population_dict = {}

    for i in range(len(county_list)):
        population_dict[county_list[i]] = population_list[i]

    with open("NY_Covid19_Data.csv") as f:
        reader = csv.reader(f, delimiter = ",", quotechar = '"')
        readerlist = list(reader)

    for county in readerlist:
        if county[0] in population_dict:
            county.append(population_dict[county[0]])
        else:
            county.append(np.nan)

    with open("NY_Covid19_Data.csv", "w") as fout:
        writer = csv.writer(fout, lineterminator = "\n")
        writer.writerows(readerlist)

    pprint(readerlist)
    print()

    return 'Data has been exported to "NY_Covid19_Data.csv"'


############ Function Call ############
pprint(web_parser4("https://www.indexmundi.com/facts/united-states/quick-facts/new-york/population#table"))

#Inconsistency Revisions
 **If you were requested to revise your inconsistency section from Phase I, enter your responses here. Otherwise, ignore this section.**

For each inconsistency (NaN, null, duplicate values, empty strings, etc.) you discover in your datasets, write at least 2 sentences stating the significance, how you identified it, and how you handled it.

1. 

2. 

3. 

4. (if applicable)

5. (if applicable)


## Data Sources

Include sources (as links) to your datasets.

*   Downloaded Dataset Source:
*   Web Collection #1 Source:
*   Web Collection #2 Source:



# Data Analysis
For the Data Analysis section, you are required to utilize your data to complete the following:

*   Create at least 5 insights
*   Generate at least 3 data visualizations
*   Export aggregated data to at least 2 summary files 

Create a function for each of the following sections mentioned above. Do not forget to fill out the explanation section for each function. 

Make sure your data analysis is not too simple. Performing complex aggregation and using modules not taught in class shows effort, which will increase the chance of receiving full credit. 

# Graphical User Interface (GUI) Implementation
If you decide to create a GUI for Phase II, please create a separate Python file (.py) to build your GUI. You must submit both the completed PhaseII.ipynb and your Python GUI file.

## Insights

In [ ]:
import pandas as pd; from pprint import pprint; import numpy as np

def insight1(filename):

    df = pd.read_csv(filename, delimiter = ",", index_col = 0)
    q1 = df.iloc[0:14,:]
    q2 = df.iloc[14:28,:]
    q3 = df.iloc[28:42,:]
    q4 = df.iloc[42:, :]

    q1_counties = list(q1.index.values)
    q2_counties = list(q2.index.values)
    q3_counties = list(q3.index.values)
    q4_counties = list(q4.index.values)

    covid_df = pd.read_csv("NY_Covid19_Data.csv", delimiter = ",", index_col = 0)

    q1_data = []
    q2_data = []
    q3_data = []
    q4_data = []

    for county in q1_counties:
            q1_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    for county in q2_counties:
            q2_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    for county in q3_counties:
            q3_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    for county in q4_counties:
            q4_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    q1.loc[:, "Average Number of Cases per Day"] = q1_data
    q2.loc[:, "Average Number of Cases per Day"] = q2_data
    q3.loc[:, "Average Number of Cases per Day"] = q3_data
    q4.loc[:, "Average Number of Cases per Day"] = q4_data

    q1.loc["Average"] = q1.mean(axis=0).round(2)
    q2.loc["Average"] = q2.mean(axis=0).round(2)
    q3.loc["Average"] = q3.mean(axis=0).round(2)
    q4.loc["Average"] = q4.mean(axis=0).round(2)

    return q1, q2, q3, q4


############ Function Call ############
pprint(insight1("income_education.csv"))

FileNotFoundError: ignored

### Insight 1 Explanation

The following code returns 4 dataframes: q1, q2, q3, and q4. The dataframes are categorized into quartiles in which q1 has data from counties with the highest median income and q4 contains the data from counties with the lowest median income. This code reads data from our summary file, "income_education.csv," and combines it with data from our other summary file, "NY_Covid19_Data.csv". We then created a new row that contains averages for each column: "Median Income", "Percent of Population with High School Diplomas", "Percent of Population with Bachelor's Degrees", and "Average Number of Cases Per Day." The insight we gained from this was that the lower two quartiles had the highest average number of cases per day while q2 had an extremely low number of cases per day. Q1, surprisingly, had a higher average number of daily cases than q4. Through all this data, we concluded that q1, the highest median income quartile, consists of families that earn enough to combat COVID-19 and are more likely to go to bars and restaruants because they have money. Q2, consisting of counties with middle class to upper middle class income, probably had the lowest infectivity because they have the resources to combat COVID-19 if someone were to get infected but most likely followed quarantine guidelines to not get infected in the first place. Q3 and Q4 most likely had the highest infectivity rates because the lower income families were probably going to work still for income resulting in higher infectivity. In addition, the lower income families probably don't have the appropriate resources for prevention.

In [ ]:
def insight2():
  
    df=pd.read_csv("NY_Covid_Deaths.csv")
    data=df[["All Ages","0-17 years","18-64 years","65+ years"]]
    data["Week"]=range(0,38)
    data.loc["Total Deaths",:]=data.sum(axis=0)
    data.to_csv("Age Group Deaths.csv", index = True)

    return data






############ Function Call ############
insight2()

### Insight 2 Explanation

The following code returns a dataframe from our data_parser2 file from phase 1 which has COVID-19 related deaths based on 4 distinct age groups: all ages, 0-17 years,18-64 years, and 65+ years. There is an additional column representing which week the data is from (February 1 to October 17). The very last row represents the total COVID-19 deaths for each age group. There are 25,226 COVID-19 deaths for people over the age of 65 years old which is over 3.5 times the amount of deaths for the age group 18-64 years (7585 deaths). The total amount of deaths are 32861 across all age groups.There were zero deaths reported for anyone under the age of 18.Looking at the data overall, we are able to see the deaths occured after week 7. The COVID-19 deaths seem to increase from weeks 7-16 and gradually decrease after week 16. 

In [ ]:
import pandas as pd; from pprint import pprint; import numpy as np

def insight3(filename):

    df = pd.read_csv(filename, delimiter = ",", index_col = 0)
    df.sort_values(by = "Percent of Population with Bachelor's Degree", ascending = False, inplace = True)


    q1 = df.iloc[0:14,:]
    q2 = df.iloc[14:28,:]
    q3 = df.iloc[28:42,:]
    q4 = df.iloc[42:, :]

    q1_counties = list(q1.index.values)
    q2_counties = list(q2.index.values)
    q3_counties = list(q3.index.values)
    q4_counties = list(q4.index.values)

    covid_df = pd.read_csv("NY_Covid19_Data.csv", delimiter = ",", index_col = 0)

    q1_data = []
    q2_data = []
    q3_data = []
    q4_data = []

    for county in q1_counties:
            q1_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    for county in q2_counties:
            q2_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    for county in q3_counties:
            q3_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    for county in q4_counties:
            q4_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    q1.loc[:, "Average Number of Cases per Day"] = q1_data
    q2.loc[:, "Average Number of Cases per Day"] = q2_data
    q3.loc[:, "Average Number of Cases per Day"] = q3_data
    q4.loc[:, "Average Number of Cases per Day"] = q4_data

    q1.loc["Average"] = q1.mean(axis=0).round(2)
    q2.loc["Average"] = q2.mean(axis=0).round(2)
    q3.loc["Average"] = q3.mean(axis=0).round(2)
    q4.loc["Average"] = q4.mean(axis=0).round(2)

    return q1, q2, q3, q4






############ Function Call ############
pprint(insight3("income_education.csv"))

### Insight 3 Explanation

Insight 3 returns four dataframes similar to insight 1. The csv file "NY_Covid19_Data.csv" was transformed into a pandas dataframe and then sorted by "Percent of Population with Bachelor's Degree" in order to create the quartiles where q1 contains the counties with the highest percentages of the population with bachelor's degrees and the q4 contains the lowest percentages of the population with bachelors degrees. We wanted to see if there was a correlation between education (specifically those with college degrees) and infectivity. Each of the four dataframes that are returned contain a final row that shows the averages for each column (specifically the average percentage of the population that have bachelor's degrees and the average number of daily COVID-19 cases for each quartile. Through this code, we saw that the lower two quartiles (q3 and q4) had a higher number of daily COVID-19 cases while the upper two quartiles (q1 and q2) had lower number of daily cases. Similar to insight 1, q1 had a higher average number of daily cases than q4 and q2 had the lowest average number of daily cases.

In [ ]:
import pandas as pd; from pprint import pprint; import numpy as np

def insight4(filename):

    df = pd.read_csv(filename, delimiter = ",", index_col = 0)
    new_df = df[["Population", "Average Number of Cases per Day"]]
    new_df = new_df.dropna()

    new_df = new_df.sort_values(by = "Population", ascending = False, inplace = False)

    q1 = new_df.iloc[0:14,:]
    q2 = new_df.iloc[14:28,:]
    q3 = new_df.iloc[28:42,:]
    q4 = new_df.iloc[42:, :]

    q1.loc["Average"] = q1.mean(axis=0).round(2)
    q2.loc["Average"] = q2.mean(axis=0).round(2)
    q3.loc["Average"] = q3.mean(axis=0).round(2)
    q4.loc["Average"] = q4.mean(axis=0).round(2)

    return q1, q2, q3, q4


############ Function Call ############
pprint(insight4("NY_Covid19_Data.csv"))

### Insight 4 Explanation

Insight 4 returns four dataframes that consist of the population and average number of cases per day for each county separated into quartiles, where q1 is the top 25% most populated counties in New York and q4 is the bottom 25% of the most populated counties in New York. Each of the 4 dataframes have an additional row that contains the averages of each column. Through this table, we can see that the most populated counties have a higher number of daily COVID-19 cases and the lower populated counties have a lower number of COVID-19 cases. We can see a direct correlation between the population and average number of daily cases so we can assume that people living in more populated areas are more likely to contract COVID-19.

In [ ]:
def insight5():
  df=pd.read_csv("NY_Covid19_Data.csv")
  df.drop(["Date of First Case","Date of Last Case","Average Number of Cases per Day","nan"],axis=1,inplace=True)
  df.loc[:,"% of People Tested Positive"]=round((df["Total Cases"]/df["Total Tests Performed"])*100,2)
  df.sort_values(by="% of People Tested Positive",ascending=False,inplace=True)
  df.loc["Sum","Total Cases"]=df["Total Cases"].sum(axis=0)
  df.loc["Sum","Total Tests Performed"]=df["Total Tests Performed"].sum(axis=0)
  df.loc["Average","% of People Tested Positive"]=(df.loc["Sum","Total Cases"]/df.loc["Sum","Total Tests Performed"])*100
  df.to_csv("% of People Tested Positive.csv", index = True)

  return df





############ Function Call ############
insight5()

### Insight 5 Explanation

The insight 5 returns a csv file made from a dataframe which contains information about County, Population, Total Cases,and Total Tests Performed from March to October sorted by % of People who tested positive (column added) in the State of New York. From the exported csv file, we are able to see that Rensselaer country had the highest percentage of people who tested positive at 7.84 percent. The country with lowest positive percentage is Tioga at only 0.15 percent. There is an additional row called "average" which tells out of all the test performed 3.59% percent of people tested positive in the state of New York. In the real world, it tell us that if you are going to get tested you need to take additional precautions at testing centers as there are 3.59 of people with COVID-19. 

## Data Visualizations

In [ ]:
import pandas as pd; from pprint import pprint; import numpy as np; import matplotlib.pyplot as plt
def visual1():

    df=pd.read_csv("NY_Covid_Deaths.csv")
    data=df[["All Ages","0-17 years","18-64 years","65+ years"]]
    data["Week"]=range(0,38)


    ax1=plt.gca()
    data.plot(kind='line',x='Week',y='65+ years',color="blue",ax=ax1,title="COVID-19 Deaths by Age Group",xlabel="Week Number",ylabel="Deaths")
    data.plot(kind='line',x='Week',y='18-64 years',color="red",ax=ax1)
    data.plot(kind='line',x='Week',y='0-17 years',color="green",ax=ax1)
    data.plot(kind='line',x='Week',y='All Ages',color="black",ax=ax1)
    plt.savefig("COVID-19 Death by Age Group Line Graph")
    plt.show()






############ Function Call ############
visual1()

### Visualization 1 Explanation

I created a 4 line graphs on a single plot to see a trend between COVID-19 deaths among different age groups over time from Februrary 1st to October 17th in the state of New York. Looking at line graph, it is evident that the first deaths occurred around week 7 for all age groups. Progessively, the death rates grew exponentially as more people were exposed to the deadly virus up until week 10. As people started taking more precautions by social distancing and wearing face coverings, the death rate dropped significantly from week 10 to week 21. The deaths became minimal afterwards. In terms of age groups, people older than 65 accounted for the greatest amount of deaths per week reaching upwards of 6,500 deaths per week. People from ages 18-64 were the second most affected age group reaching the highs of 5000 deaths per week. Ages 18 and under, 0 people died according to our data. 

In [ ]:
import pandas as pd; from pprint import pprint; import numpy as np; import matplotlib.pyplot as plt

def visual2(filename):

    df = pd.read_csv(filename, delimiter = ",", index_col = 0)
    df.sort_values(by = "Percent of Population with Bachelor's Degree", ascending = False, inplace = True)


    q1 = df.iloc[0:14,:]
    q2 = df.iloc[14:28,:]
    q3 = df.iloc[28:42,:]
    q4 = df.iloc[42:, :]

    q1_counties = list(q1.index.values)
    q2_counties = list(q2.index.values)
    q3_counties = list(q3.index.values)
    q4_counties = list(q4.index.values)

    covid_df = pd.read_csv("NY_Covid19_Data.csv", delimiter = ",", index_col = 0)

    q1_data = []
    q2_data = []
    q3_data = []
    q4_data = []

    for county in q1_counties:
            q1_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    for county in q2_counties:
            q2_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    for county in q3_counties:
            q3_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    for county in q4_counties:
            q4_data.append(covid_df.loc[county, "Average Number of Cases per Day"])

    q1.loc[:, "Average Number of Cases per Day"] = q1_data
    q2.loc[:, "Average Number of Cases per Day"] = q2_data
    q3.loc[:, "Average Number of Cases per Day"] = q3_data
    q4.loc[:, "Average Number of Cases per Day"] = q4_data

    q1.loc["Average"] = q1.mean(axis=0).round(2)
    q2.loc["Average"] = q2.mean(axis=0).round(2)
    q3.loc["Average"] = q3.mean(axis=0).round(2)
    q4.loc["Average"] = q4.mean(axis=0).round(2)

    data = [q1.loc["Average"].loc["Average Number of Cases per Day"], q2.loc["Average"].loc["Average Number of Cases per Day"], q3.loc["Average"].loc["Average Number of Cases per Day"], q4.loc["Average"].loc["Average Number of Cases per Day"]]

    graph_df = pd.DataFrame({"Average Number of Cases per Day" : data, "Quartile" : ["Q1", "Q2", "Q3", "Q4"]})

    graph_df.plot(kind = 'bar', x = 'Quartile', y = 'Average Number of Cases per Day', color = 'red')

    plt.xlabel("Quartile")
    plt.ylabel("Average Number of Cases per Day")
    plt.title("College Education vs Average Number of Cases per Day by County Quartile")
    plt.show()



############ Function Call ############
pprint(visual2("income_education.csv"))

### Visualization 2 Explanation

Visual2 returns a bar graph where each bar represents a portion of the population that has a Bachelor's Degree. Each portion, a quartile, contains the average of roughly 14 counties in New York and shows the average daily cases for those counties. Q1 represents the top 25% of counties with the highest percentage of population with Bachelor's Degrees and Q4 represents the lowest. Through this visualization we can see that the lower two quartiles tend to have higher average number of daily cases while the upper two quartiles have less. Q1, however, showed a suprisingly high number of daily average cases. This could be the result of many other factors, but in this cases we assumed that those who have college degrees tend to have more money and therefore are probably spending more time going out during quarantine because they have the resources to do so (similar to insight 1).

In [ ]:
import pandas as pd; from pprint import pprint; import numpy as np; import matplotlib.pyplot as plt

def visual3(filename):

    df = pd.read_csv(filename, delimiter = ",", index_col = 0)
    new_df = df[["Population", "Average Number of Cases per Day"]]
    new_df = new_df.dropna()

    new_df = new_df.sort_values(by = "Population", ascending = False, inplace = False)

    q1 = new_df.iloc[0:14,:]
    q2 = new_df.iloc[14:28,:]
    q3 = new_df.iloc[28:42,:]
    q4 = new_df.iloc[42:, :]

    q1.loc["Average"] = q1.mean(axis=0).round(2)
    q2.loc["Average"] = q2.mean(axis=0).round(2)
    q3.loc["Average"] = q3.mean(axis=0).round(2)
    q4.loc["Average"] = q4.mean(axis=0).round(2)

    data = [q1.loc["Average"].loc["Average Number of Cases per Day"], q2.loc["Average"].loc["Average Number of Cases per Day"], q3.loc["Average"].loc["Average Number of Cases per Day"], q4.loc["Average"].loc["Average Number of Cases per Day"]]

    graph_df = pd.DataFrame({"Average Number of Cases per Day" : data, "Quartile" : ["Q1", "Q2", "Q3", "Q4"]})

    graph_df.plot(kind = 'bar', x = 'Quartile', y = 'Average Number of Cases per Day', color = 'blue')

    plt.xlabel("Quartile")
    plt.ylabel("Average Number of Cases per Day")
    plt.title("Population vs Average Number of Cases per Day by County Quartile")

    plt.show()


############ Function Call ############
pprint(visual3("NY_Covid19_Data.csv"))

### Visualization 3 Explanation

Visual3 returns a bar graph that, again, separates the counties into different quartiles by population. Q1 represents the top 25% most populated counties and Q4 represents the bottom 25% most populated counties. It is much more obvious through this bar graph that a trend exists where highly populated counties tend to have a higher number of average daily cases while the least populated counties tend to have a lower number of average daily cases.

## Summary Files

In [ ]:
import csv; from pprint import pprint

def summary1(filename):
    with open(filename) as f:
        reader=csv.reader(f,delimiter=",",quotechar='"')
        reader_list=list(reader)

        return reader_list



############ Function Call ############
pprint(summary1("NY_Covid19_Data.csv"))

In [ ]:
import csv; from pprint import pprint

def summary2(filename):
    with open(filename) as f:
        reader=csv.reader(f,delimiter=",",quotechar='"')
        reader_list=list(reader)

        return reader_list



############ Function Call ############
pprint(summary2("NY_Covid_Deaths.csv"))

In [ ]:
import csv; from pprint import pprint

def summary3(filename):
    with open(filename) as f:
        reader=csv.reader(f,delimiter=",",quotechar='"')
        reader_list=list(reader)

        return reader_list


############ Function Call ############
pprint(summary3("income_education.csv"))

# Cited Sources

If you used any additional sources to complete your Data Analysis section, list them here:


*   Example Module Documentation
*   Example Stack Overflow Assistance



# Video Presentation

If you uploaded your Video Presentation to Bluejeans, YouTube, or any other streaming services, please provide the link here:


*   https://bluejeans.com/s/NxnL@YRKSqt


Make sure the video sharing permissions are accessible for anyone with the provided link.